In [5]:
import findspark
findspark.init('/usr/local/spark')

In [6]:
import pyspark

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("check_point_2_") \
        .config("spark.sql.warehouse.dir", "hdfs://localhost:54310/user/hive/warehouse") \
        .enableHiveSupport() \
        .getOrCreate()

In [69]:
df = spark.read.csv("Checkpoint_1_merged.csv",header=True,inferSchema=True)

# Q-1

In [10]:
spark.sql("show databases").show()

+------------+
|databaseName|
+------------+
|     default|
|    deloitte|
+------------+



In [ ]:
spark.sql("create database HR_data")

In [ ]:
spark.sql("show databases").show()

In [ ]:
spark.sql("use hr_data")

In [ ]:
spark.sql("show tables").show()

In [ ]:
df.write.format("parquet").saveAsTable("hr_details")

In [9]:
spark.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



 # Q-2

In [ ]:
type(df)

In [ ]:
df.show()

In [ ]:
df.columns

In [ ]:
df.printSchema()

In [12]:
df.describe().show()

+-------+------------------+-----------------+------------+------------------------+------------------+------------+----------------------------+---------------------------+----------------------+-------------+-------------------------+------+-----------------+------------------+-----------+------------------+------------------+------------------+-----------+--------------+---------+---------+----------+-----------------+----+
|summary|               Sno|    Candidate_Ref|DOJ_Extended|Duration_to_accept_offer|     Notice_Period|Offered_Band|Percent_hike_expected_in_CTC|Percent_hike_offered_in_CTC|Percent_difference_CTC|Joining_Bonus|Candidate_relocate_actual|Gender| Candidate_Source|        Rex_in_Yrs|Location_ID|       Postal_Code|               Age|            LOB_Id|Domicile_ID| Domicile_Name| Location|Region_ID|    Status|Date_of_Interview| LOB|
+-------+------------------+-----------------+------------+------------------------+------------------+------------+----------------------

# Q-3

In [70]:
from pyspark.sql.functions import col,isnan, when, count

In [71]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()

+---+-------------+------------+------------------------+-------------+------------+----------------------------+---------------------------+----------------------+-------------+-------------------------+------+----------------+----------+-----------+-----------+---+------+-----------+-------------+--------+---------+------+-----------------+---+
|Sno|Candidate_Ref|DOJ_Extended|Duration_to_accept_offer|Notice_Period|Offered_Band|Percent_hike_expected_in_CTC|Percent_hike_offered_in_CTC|Percent_difference_CTC|Joining_Bonus|Candidate_relocate_actual|Gender|Candidate_Source|Rex_in_Yrs|Location_ID|Postal_Code|Age|LOB_Id|Domicile_ID|Domicile_Name|Location|Region_ID|Status|Date_of_Interview|LOB|
+---+-------------+------------+------------------------+-------------+------------+----------------------------+---------------------------+----------------------+-------------+-------------------------+------+----------------+----------+-----------+-----------+---+------+-----------+-------------+--

In [72]:
df_Columns=['Offered_Band','Age','Location','Region_ID']
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_Columns]
   ).show()

+------------+---+--------+---------+
|Offered_Band|Age|Location|Region_ID|
+------------+---+--------+---------+
|          13| 15|      13|       13|
+------------+---+--------+---------+



In [16]:
df.createOrReplaceTempView("mydata")

In [17]:
spark.sql("select * from mydata").show()

+---+-------------+------------+------------------------+-------------+------------+----------------------------+---------------------------+----------------------+-------------+-------------------------+------+-----------------+----------+-----------+-----------+---+------+-----------+----------------+---------+---------+----------+-----------------+----------+
|Sno|Candidate_Ref|DOJ_Extended|Duration_to_accept_offer|Notice_Period|Offered_Band|Percent_hike_expected_in_CTC|Percent_hike_offered_in_CTC|Percent_difference_CTC|Joining_Bonus|Candidate_relocate_actual|Gender| Candidate_Source|Rex_in_Yrs|Location_ID|Postal_Code|Age|LOB_Id|Domicile_ID|   Domicile_Name| Location|Region_ID|    Status|Date_of_Interview|       LOB|
+---+-------------+------------+------------------------+-------------+------------+----------------------------+---------------------------+----------------------+-------------+-------------------------+------+-----------------+----------+-----------+-----------+---+--

In [73]:
from pyspark.sql.functions import mean,round

In [ ]:
[df.groupby(i).count().orderBy("count", ascending=False).first()[0] for i in df.columns].show()

AGE

In [74]:
df.select('Age').show()

+---+
|Age|
+---+
| 34|
| 34|
| 27|
| 34|
| 34|
| 34|
| 32|
| 34|
| 26|
| 34|
| 34|
| 34|
| 29|
| 34|
| 34|
| 34|
| 36|
| 34|
| 34|
| 34|
+---+
only showing top 20 rows



In [75]:
df_Columns=["Age"]
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_Columns]
   ).show()

+---+
|Age|
+---+
| 15|
+---+



In [76]:
mean_age= df.select(round(mean(df['Age']))).collect()

In [24]:
mean_age

[Row(round(avg(Age), 0)=30.0)]

In [77]:
df=df.na.fill(mean_age[0][0], subset=['Age'])

In [78]:
df_Columns=["Age"]
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_Columns]
   ).show()

+---+
|Age|
+---+
|  0|
+---+



OFFER BAND

In [79]:
df_Columns=["Offered_Band"]
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_Columns]
   ).show()

+------------+
|Offered_Band|
+------------+
|          13|
+------------+



In [80]:
mode_offered_band=df.groupby("Offered_Band").count().orderBy("count", ascending=False).first()[0]

In [81]:
df=df.na.fill(mode_offered_band[0][0], subset=['Offered_Band'])

In [82]:
df_Columns=["Offered_Band"]
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_Columns]
   ).show()

+------------+
|Offered_Band|
+------------+
|           0|
+------------+



LOCATION

In [83]:
df_Columns=["Location"]
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_Columns]
   ).show()

+--------+
|Location|
+--------+
|      13|
+--------+



In [84]:
mode_location=df.groupby("Location").count().orderBy("count", ascending=False).first()[0]
df=df.na.fill(mode_location[0][0], subset=['Location'])

In [85]:
df_Columns=["Location"]
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_Columns]
   ).show()

+--------+
|Location|
+--------+
|       0|
+--------+



In [34]:
REGION ID

SyntaxError: invalid syntax (<ipython-input-34-089145182910>, line 1)

In [86]:
df_Columns=["Region_ID"]
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_Columns]
   ).show()

+---------+
|Region_ID|
+---------+
|       13|
+---------+



In [87]:
mode_region=df.groupby("Region_ID").count().orderBy("count", ascending=False).first()[0]
df=df.na.fill(mode_location[0][0], subset=['Region_ID'])

In [88]:
df_Columns=["Region_ID"]
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_Columns]
   ).show()

+---------+
|Region_ID|
+---------+
|        0|
+---------+



AFTER DATA CLEANING

In [89]:
df_Columns=['Offered_Band','Age','Location','Region_ID']
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_Columns]
   ).show()

+------------+---+--------+---------+
|Offered_Band|Age|Location|Region_ID|
+------------+---+--------+---------+
|           0|  0|       0|        0|
+------------+---+--------+---------+



# Q-4

In [90]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

In [91]:
from pyspark.ml import Pipeline

In [92]:
df.columns

['Sno',
 'Candidate_Ref',
 'DOJ_Extended',
 'Duration_to_accept_offer',
 'Notice_Period',
 'Offered_Band',
 'Percent_hike_expected_in_CTC',
 'Percent_hike_offered_in_CTC',
 'Percent_difference_CTC',
 'Joining_Bonus',
 'Candidate_relocate_actual',
 'Gender',
 'Candidate_Source',
 'Rex_in_Yrs',
 'Location_ID',
 'Postal_Code',
 'Age',
 'LOB_Id',
 'Domicile_ID',
 'Domicile_Name',
 'Location',
 'Region_ID',
 'Status',
 'Date_of_Interview',
 'LOB']

Convert the platform columns to numerical

In [93]:
import pandas as pd

In [94]:
df=df.toPandas()

In [95]:
categoricalColumns = ['DOJ_Extended','Joining_Bonus','Candidate_relocate_actual','Gender','Candidate_Source','LOB']

In [101]:
dummies=pd.get_dummies(df[['DOJ_Extended','Joining_Bonus','Candidate_relocate_actual','Gender','Candidate_Source','LOB','Offered_Band']])

In [102]:
dummies

,DOJ_Extended_No,DOJ_Extended_Yes,Joining_Bonus_No,Joining_Bonus_Yes,Candidate_relocate_actual_No,Candidate_relocate_actual_Yes,Gender_Female,Gender_Male,Candidate_Source_Agency,Candidate_Source_Direct,...,LOB_ERS,LOB_ETS,LOB_Healthcare,LOB_INFRA,LOB_MMS,Offered_Band_E,Offered_Band_E0,Offered_Band_E1,Offered_Band_E2,Offered_Band_E3
0,0,1,1,0,1,0,1,0,1,0,...,1,0,0,0,0,0,0,0,1,0
1,1,0,1,0,1,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
2,1,0,1,0,1,0,0,1,1,0,...,0,0,0,1,0,0,0,0,1,0
3,1,0,1,0,1,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
4,0,1,1,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
5,0,1,1,0,1,0,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0
6,0,1,1,0,1,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
7,0,1,1,0,1,0,1,0,0,1,...,0,0,1,0,0,0,0,1,0,0
8,1,0,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9,1,0,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [121]:
df1=df.copy()

In [122]:
df1.drop(['DOJ_Extended','Joining_Bonus','Candidate_relocate_actual','Gender','Candidate_Source','LOB','Offered_Band'], axis=1, inplace= True)

In [123]:
df1

,Sno,Candidate_Ref,Duration_to_accept_offer,Notice_Period,Percent_hike_expected_in_CTC,Percent_hike_offered_in_CTC,Percent_difference_CTC,Rex_in_Yrs,Location_ID,Postal_Code,Age,LOB_Id,Domicile_ID,Domicile_Name,Location,Region_ID,Status,Date_of_Interview
0,1,2110407,14,30,-20.79,13.16,42.86,7,L001,201301,34,1,D6,Goa,Noida,R1,Joined,1/8/2013
1,2,2112635,18,30,50.00,320.00,180.00,8,L002,600020,34,2,D9,Himachal Pradesh,Chennai,R2,Joined,1/12/2013
2,3,2112838,3,45,42.84,42.84,0.00,4,L001,201301,27,2,D10,Jharkhand,Noida,R1,Joined,1/25/2013
3,4,2115021,26,30,42.84,42.84,0.00,4,L001,201301,34,2,D3,Chandigarh,Noida,R1,Joined,1/27/2013
4,5,2115125,1,120,42.59,42.59,0.00,6,L001,201301,34,2,D21,Tripura,Noida,R1,Joined,1/28/2013
5,6,2117167,17,30,42.83,42.83,0.00,2,L001,201301,34,2,D24,West Bengal,Noida,R1,Joined,2/3/2013
6,7,2119124,37,30,31.58,31.58,0.00,7,L001,201301,32,2,D23,Uttarkhand,Noida,R1,Joined,2/11/2013
7,9,2127572,16,0,-20.00,-20.00,0.00,8,L001,201301,34,3,D6,Goa,Noida,R1,Joined,2/16/2013
8,11,2138169,1,30,-22.22,-22.22,0.00,3,L003,122107,26,4,D5,Delhi,Gurgaon,R1,Joined,2/16/2013
9,12,2143362,6,30,240.00,220.00,-5.88,3,L002,600020,34,5,D23,Uttarkhand,Chennai,R2,Joined,3/7/2013


In [124]:
df1= pd.concat([df1, dummies], axis=1)
df1

,Sno,Candidate_Ref,Duration_to_accept_offer,Notice_Period,Percent_hike_expected_in_CTC,Percent_hike_offered_in_CTC,Percent_difference_CTC,Rex_in_Yrs,Location_ID,Postal_Code,...,LOB_ERS,LOB_ETS,LOB_Healthcare,LOB_INFRA,LOB_MMS,Offered_Band_E,Offered_Band_E0,Offered_Band_E1,Offered_Band_E2,Offered_Band_E3
0,1,2110407,14,30,-20.79,13.16,42.86,7,L001,201301,...,1,0,0,0,0,0,0,0,1,0
1,2,2112635,18,30,50.00,320.00,180.00,8,L002,600020,...,0,0,0,1,0,0,0,0,1,0
2,3,2112838,3,45,42.84,42.84,0.00,4,L001,201301,...,0,0,0,1,0,0,0,0,1,0
3,4,2115021,26,30,42.84,42.84,0.00,4,L001,201301,...,0,0,0,1,0,0,0,0,1,0
4,5,2115125,1,120,42.59,42.59,0.00,6,L001,201301,...,0,0,0,1,0,0,0,0,1,0
5,6,2117167,17,30,42.83,42.83,0.00,2,L001,201301,...,0,0,0,1,0,0,0,1,0,0
6,7,2119124,37,30,31.58,31.58,0.00,7,L001,201301,...,0,0,0,1,0,0,0,0,1,0
7,9,2127572,16,0,-20.00,-20.00,0.00,8,L001,201301,...,0,0,1,0,0,0,0,1,0,0
8,11,2138169,1,30,-22.22,-22.22,0.00,3,L003,122107,...,0,0,0,0,0,0,0,1,0,0
9,12,2143362,6,30,240.00,220.00,-5.88,3,L002,600020,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
#Dsiplay the categorial column and numerical column

In [125]:
df1.columns

Index(['Sno', 'Candidate_Ref', 'Duration_to_accept_offer', 'Notice_Period',
       'Percent_hike_expected_in_CTC', 'Percent_hike_offered_in_CTC',
       'Percent_difference_CTC', 'Rex_in_Yrs', 'Location_ID', 'Postal_Code',
       'Age', 'LOB_Id', 'Domicile_ID', 'Domicile_Name', 'Location',
       'Region_ID', 'Status', 'Date_of_Interview', 'DOJ_Extended_No',
       'DOJ_Extended_Yes', 'Joining_Bonus_No', 'Joining_Bonus_Yes',
       'Candidate_relocate_actual_No', 'Candidate_relocate_actual_Yes',
       'Gender_Female', 'Gender_Male', 'Candidate_Source_Agency',
       'Candidate_Source_Direct', 'Candidate_Source_Employee Referral',
       'LOB_AXON', 'LOB_BFSI', 'LOB_CSMP', 'LOB_EAS', 'LOB_ERS', 'LOB_ETS',
       'LOB_Healthcare', 'LOB_INFRA', 'LOB_MMS', 'Offered_Band_E',
       'Offered_Band_E0', 'Offered_Band_E1', 'Offered_Band_E2',
       'Offered_Band_E3'],
      dtype='object')

In [ ]:
df1['Status']

In [127]:
df1['Status']= np.where(df['Status']=='Joined',1,0)

In [ ]:
df1['Status']

In [133]:
final=spark.createDataFrame(df1)

In [138]:
final.printSchema()

root
 |-- Sno: long (nullable = true)
 |-- Candidate_Ref: long (nullable = true)
 |-- Duration_to_accept_offer: long (nullable = true)
 |-- Notice_Period: long (nullable = true)
 |-- Percent_hike_expected_in_CTC: double (nullable = true)
 |-- Percent_hike_offered_in_CTC: double (nullable = true)
 |-- Percent_difference_CTC: double (nullable = true)
 |-- Rex_in_Yrs: long (nullable = true)
 |-- Location_ID: string (nullable = true)
 |-- Postal_Code: long (nullable = true)
 |-- Age: long (nullable = true)
 |-- LOB_Id: long (nullable = true)
 |-- Domicile_ID: string (nullable = true)
 |-- Domicile_Name: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Region_ID: string (nullable = true)
 |-- Status: long (nullable = true)
 |-- Date_of_Interview: string (nullable = true)
 |-- DOJ_Extended_No: long (nullable = true)
 |-- DOJ_Extended_Yes: long (nullable = true)
 |-- Joining_Bonus_No: long (nullable = true)
 |-- Joining_Bonus_Yes: long (nullable = true)
 |-- Candidate_rel

In [178]:
final.show(1)

+---+-------------+------------------------+-------------+----------------------------+---------------------------+----------------------+----------+-----------+-----------+---+------+-----------+-------------+--------+---------+------+-----------------+---------------+----------------+----------------+-----------------+----------------------------+-----------------------------+-------------+-----------+-----------------------+-----------------------+----------------------------------+--------+--------+--------+-------+-------+-------+--------------+---------+-------+--------------+---------------+---------------+---------------+---------------+
|Sno|Candidate_Ref|Duration_to_accept_offer|Notice_Period|Percent_hike_expected_in_CTC|Percent_hike_offered_in_CTC|Percent_difference_CTC|Rex_in_Yrs|Location_ID|Postal_Code|Age|LOB_Id|Domicile_ID|Domicile_Name|Location|Region_ID|Status|Date_of_Interview|DOJ_Extended_No|DOJ_Extended_Yes|Joining_Bonus_No|Joining_Bonus_Yes|Candidate_relocate_actua

In [255]:
df_assembler = VectorAssembler(inputCols=['Duration_to_accept_offer', 'Notice_Period',
       'Percent_hike_expected_in_CTC', 'Percent_hike_offered_in_CTC',
       'Rex_in_Yrs',
       'Age', 'LOB_Id', 'DOJ_Extended_No',
       'DOJ_Extended_Yes', 'Joining_Bonus_No', 
       'Candidate_relocate_actual_No', 'Candidate_relocate_actual_Yes',
        'Gender_Male', 
       'Candidate_Source_Direct', 'Candidate_Source_Employee Referral',
        'LOB_BFSI', 'LOB_CSMP', 'LOB_EAS', 'LOB_ERS', 'LOB_ETS',
       'LOB_Healthcare', 'LOB_INFRA', 'LOB_MMS', 'Offered_Band_E',
        'Offered_Band_E1', 'Offered_Band_E2',
       'Offered_Band_E3'], outputCol="features")


In [256]:
"""df_assembler = VectorAssembler(inputCols=['Duration_to_accept_offer', 'Notice_Period',
       'Rex_in_Yrs',
       'Age', 'LOB_Id', 'DOJ_Extended_No',
       'DOJ_Extended_Yes', 'Joining_Bonus_No', 
       'Candidate_relocate_actual_No', 'Candidate_relocate_actual_Yes',
        'Gender_Male'], outputCol="features")"""

'df_assembler = VectorAssembler(inputCols=[\'Duration_to_accept_offer\', \'Notice_Period\',\n       \'Rex_in_Yrs\',\n       \'Age\', \'LOB_Id\', \'DOJ_Extended_No\',\n       \'DOJ_Extended_Yes\', \'Joining_Bonus_No\', \n       \'Candidate_relocate_actual_No\', \'Candidate_relocate_actual_Yes\',\n        \'Gender_Male\'], outputCol="features")'

In [257]:
from pyspark.ml.feature import StandardScaler

In [258]:
ss=StandardScaler(inputCol="")

In [259]:
output = df_assembler.transform(final)

In [260]:
import numpy as np

In [261]:
df2 = output.select('features','Status')

In [262]:
df2.show(40)

+--------------------+------+
|            features|Status|
+--------------------+------+
|(27,[0,1,2,3,4,5,...|     1|
|(27,[0,1,2,3,4,5,...|     1|
|(27,[0,1,2,3,4,5,...|     1|
|(27,[0,1,2,3,4,5,...|     1|
|(27,[0,1,2,3,4,5,...|     1|
|(27,[0,1,2,3,4,5,...|     1|
|(27,[0,1,2,3,4,5,...|     1|
|(27,[0,2,3,4,5,6,...|     1|
|(27,[0,1,2,3,4,5,...|     1|
|(27,[0,1,2,3,4,5,...|     1|
|(27,[0,1,2,3,4,5,...|     0|
|(27,[0,2,3,4,5,6,...|     1|
|(27,[0,1,2,3,4,5,...|     0|
|(27,[0,1,2,3,4,5,...|     0|
|(27,[0,1,2,3,4,5,...|     0|
|(27,[0,1,2,3,4,5,...|     1|
|(27,[0,1,2,3,4,5,...|     0|
|(27,[0,2,3,4,5,6,...|     1|
|(27,[0,1,2,3,4,5,...|     1|
|(27,[0,1,2,3,4,5,...|     1|
|(27,[0,2,3,4,5,6,...|     1|
|(27,[1,2,3,4,5,6,...|     1|
|(27,[0,2,3,4,5,6,...|     1|
|(27,[0,1,2,3,4,5,...|     1|
|(27,[0,2,3,4,5,6,...|     1|
|(27,[0,1,2,4,5,6,...|     0|
|(27,[0,1,2,3,4,5,...|     1|
|(27,[0,1,2,3,4,5,...|     1|
|(27,[2,3,4,5,6,8,...|     0|
|(27,[0,1,2,3,4,5,...|     1|
|(27,[0,1,

In [263]:
train,test = df2.randomSplit([0.8,0.2])

In [264]:
from pyspark.ml.classification import LogisticRegression

In [265]:
log_model = LogisticRegression(labelCol='Status')

In [266]:
fitted_model = log_model.fit(train)

In [267]:
train_summary = fitted_model.summary

In [268]:
test_results = fitted_model.evaluate(test)

In [269]:
test_results.predictions.show(25)

+--------------------+------+--------------------+--------------------+----------+
|            features|Status|       rawPrediction|         probability|prediction|
+--------------------+------+--------------------+--------------------+----------+
|(27,[0,1,2,3,4,5,...|     1|[-1.4168624114454...|[0.19515392976730...|       1.0|
|(27,[0,1,2,3,4,5,...|     0|[-1.7229676940720...|[0.15148930072230...|       1.0|
|(27,[0,1,2,3,4,5,...|     1|[-1.7874332670318...|[0.14338770082445...|       1.0|
|(27,[0,1,2,3,4,5,...|     1|[-1.4576876350190...|[0.18882125021006...|       1.0|
|(27,[0,1,2,3,4,5,...|     0|[-1.3376511892617...|[0.20789658499355...|       1.0|
|(27,[0,1,2,3,4,5,...|     1|[-1.3674784335910...|[0.20302754892492...|       1.0|
|(27,[0,1,2,3,4,5,...|     0|[-0.0984904270865...|[0.47539727796082...|       1.0|
|(27,[0,1,2,3,4,5,...|     0|[-1.5071986316696...|[0.18135432692386...|       1.0|
|(27,[0,1,2,3,4,5,...|     1|[-1.4262677306556...|[0.19368088122057...|       1.0|
|(27

In [270]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [271]:
cm = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Status')

In [272]:
acc =cm.evaluate(test_results.predictions)

In [273]:
acc

0.5396982555398396

In [276]:
from pyspark.ml.classification import DecisionTreeClassifier

In [284]:
tree = DecisionTreeClassifier(rawPredictionCol='prediction_tree', labelCol='Status')

In [285]:
tree = tree.fit(train)

In [286]:
pred_tree=tree.transform(test)

In [287]:
pred_tree.show(5)

+--------------------+------+---------------+--------------------+----------+
|            features|Status|prediction_tree|         probability|prediction|
+--------------------+------+---------------+--------------------+----------+
|(27,[0,1,2,3,4,5,...|     1|   [70.0,578.0]|[0.10802469135802...|       1.0|
|(27,[0,1,2,3,4,5,...|     0|   [70.0,578.0]|[0.10802469135802...|       1.0|
|(27,[0,1,2,3,4,5,...|     1|   [70.0,578.0]|[0.10802469135802...|       1.0|
|(27,[0,1,2,3,4,5,...|     1|   [70.0,578.0]|[0.10802469135802...|       1.0|
|(27,[0,1,2,3,4,5,...|     0| [457.0,2101.0]|[0.17865519937451...|       1.0|
+--------------------+------+---------------+--------------------+----------+
only showing top 5 rows



In [291]:

acc_tree =tree.evaluate(prediction_tree)

AttributeError: 'DecisionTreeClassificationModel' object has no attribute 'evaluate'

In [274]:
categoricalColumns = ['DOJ_Extended','Joining_Bonus','Candidate_relocate_actual','Gender','Candidate_Source','LOB']

In [275]:
numericalColumns = ['Duration_to_accept_offer','Notice_Period','Percent_hike_expected_in_CTC','Percent_hike_offered_in_CTC','Percent_difference_CTC','Rex_in_Yrs','Age']